In [7]:
!pip install openai google-api-python-client google-auth-httplib2 google-auth-oauthlib langchain


In [8]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request  
import pickle
import os

# Define the scopes (permissions)
SCOPES = [
    'https://www.googleapis.com/auth/calendar.readonly',
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/documents.readonly',
]

def authenticate_google():
    creds = None
    if os.path.exists('token.pkl'):
        with open('token.pkl', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client_secret_885913184552-86iog4pkulojgr4r46vjse61ukaa3ds8.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pkl', 'wb') as token:
            pickle.dump(creds, token)
    return creds

# Authenticate
creds = authenticate_google()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=885913184552-86iog4pkulojgr4r46vjse61ukaa3ds8.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53287%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocuments.readonly&state=qqrXnpclm6aISpo8LJIxhZbLo5EKbu&access_type=offline


In [ ]:
from datetime import datetime, timedelta
import pytz
from dateutil.parser import parse
from googleapiclient.discovery import build

def list_today_calendar_events(creds):
    service = build('calendar', 'v3', credentials=creds)

    local_tz = pytz.timezone('Africa/Lagos')
    now = datetime.now(local_tz)
    start_of_day = now.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = now.replace(hour=23, minute=59, second=59, microsecond=0)

    time_min = start_of_day.isoformat()
    time_max = end_of_day.isoformat()

    events_result = service.events().list(
        calendarId='primary',
        timeMin=time_min,
        timeMax=time_max,
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    events = events_result.get('items', [])

    if not events:
        print('🌸 No events scheduled for today.')
        return

    print('📅 Today\'s Events:')
    print('-------------------')

    previous_event_end = None

    for event in events:
        summary = event.get('summary', 'No Title')
        
        if 'dateTime' in event['start']:
            start_dt = parse(event['start']['dateTime'])
            end_dt = parse(event['end']['dateTime'])
            print(f"🕒 {start_dt.strftime('%H:%M')} - {end_dt.strftime('%H:%M')} | {summary}")
        else:
            print(f"📅 All Day | {summary}")
            continue

        # Meeting Preparation Reminder
        keywords = ['call', 'meeting', 'sync', 'catch-up']
        if any(keyword.lower() in summary.lower() for keyword in keywords):
            print("   ➡️ Prep needed: Fetch notes, LinkedIn bios, previous threads.")

        # Overlap detection
        if previous_event_end and start_dt < previous_event_end:
            print("   ⚠️ Conflict detected with previous event! Consider rescheduling.")

        previous_event_end = end_dt

    print('-------------------')

# Call the function
list_today_calendar_events(creds)



In [10]:
def calendar_agent_menu(creds):
    while True:
        print("\n📅 Calendar Agent Menu:")
        print("1. View Today's Events")
        print("2. Create New Event")
        print("3. Reschedule Event")
        print("4. Delete Event")
        print("5. Exit")
        choice = input("Enter your choice (1-5): ")

        if choice == '1':
            list_today_calendar_events(creds)

        elif choice == '2':
            # Create event
            summary = input("Enter event summary/title: ")
            description = input("Enter event description (optional): ")
            attendees = input("Enter attendees' emails separated by comma (optional): ").split(',')

            date_input = input("Enter date (YYYY-MM-DD): ")
            start_time_input = input("Enter start time (HH:MM, 24hr format): ")
            end_time_input = input("Enter end time (HH:MM, 24hr format): ")

            local_tz = pytz.timezone('Africa/Lagos')
            start_dt = local_tz.localize(datetime.strptime(f"{date_input} {start_time_input}", "%Y-%m-%d %H:%M"))
            end_dt = local_tz.localize(datetime.strptime(f"{date_input} {end_time_input}", "%Y-%m-%d %H:%M"))

            create_calendar_event(creds, summary, start_dt, end_dt, description, [email.strip() for email in attendees if email])

        elif choice == '3':
            # Reschedule event
            event_id = input("Enter the Event ID to reschedule: ")

            date_input = input("Enter new date (YYYY-MM-DD): ")
            start_time_input = input("Enter new start time (HH:MM, 24hr format): ")
            end_time_input = input("Enter new end time (HH:MM, 24hr format): ")

            local_tz = pytz.timezone('Africa/Lagos')
            new_start_dt = local_tz.localize(datetime.strptime(f"{date_input} {start_time_input}", "%Y-%m-%d %H:%M"))
            new_end_dt = local_tz.localize(datetime.strptime(f"{date_input} {end_time_input}", "%Y-%m-%d %H:%M"))

            reschedule_calendar_event(creds, event_id, new_start_dt, new_end_dt)

        elif choice == '4':
            # Delete event
            event_id = input("Enter the Event ID to delete: ")
            delete_calendar_event(creds, event_id)

        elif choice == '5':
            print("👋 Exiting Calendar Agent. Have a wonderful day!")
            break

        else:
            print("⚠️ Invalid choice. Please try again.")


In [12]:
calendar_agent_menu(creds)



📅 Calendar Agent Menu:
1. View Today's Events
2. Create New Event
3. Reschedule Event
4. Delete Event
5. Exit


Enter your choice (1-5):  5


👋 Exiting Calendar Agent. Have a wonderful day!


In [14]:
!pip install python-dateutil

In [5]:
SCOPES += [
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/gmail.send',
    'https://www.googleapis.com/auth/gmail.modify',
]

NameError: name 'SCOPES' is not defined

In [3]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

def authenticate_gmail():
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)
    service = build('gmail', 'v1', credentials=creds)
    return service

gmail_service = authenticate_gmail()

NameError: name 'SCOPES' is not defined

In [1]:
import base64
import email
from googleapiclient.errors import HttpError

def list_and_summarize_emails(service, max_results=5):
    results = service.users().messages().list(userId='me', labelIds=['INBOX'], maxResults=1).execute()
    msgs = results.get('messages', [])
    if not msgs:
        print("📭 No emails found.")
        return
    for m in msgs:
        msg = service.users().messages().get(userId='me', id=m['id'], format='raw').execute()
        raw = base64.urlsafe_b64decode(msg['raw'].encode())
        mime = email.message_from_bytes(raw)
        subj = mime['subject']
        frm = mime['from']
        payload = mime.get_payload()
        body = ""
        if mime.is_multipart():
            for part in payload:
                if part.get_content_type() == 'text/plain':
                    body = part.get_payload(decode=True).decode()
                    break
        else:
            body = payload if isinstance(payload, str) else body
        snippet = body[:200].replace('\n',' ')
        print(f"✉️ From: {frm}\n    Subject: {subj}\n    Preview: {snippet}...\n---")

In [ ]:
from email.mime.text import MIMEText

def send_email(service, to, subject, message_text):
    message = MIMEText(message_text, 'plain', 'utf-8')
    message['to'] = to
    message['subject'] = subject
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    service.users().messages().send(userId='me', body={'raw': raw}).execute()
    print("✅ Email sent!")

In [ ]:
def admin_agent_menu(creds, gmail_service):
    while True:
        print("""
🔧 Admin Agent Menu:
1. View Today's Calendar
2. Create New Event
3. Reschedule Event
4. Delete Event
5. View & Summarize Emails
6. Send a New Email
7. Exit
""")
        choice = input("Enter your choice (1-7): ")
        if choice == '1':
            list_today_calendar_events(creds)
        elif choice == '2':
            # ... calendar event creation code ...
            pass
        # ...
        elif choice == '5':
            list_and_summarize_emails(gmail_service)
        elif choice == '6':
            to = input("Recipient email: ")
            subject = input("Subject: ")
            body = input("Message text: ")
            send_email(gmail_service, to, subject, body)
        elif choice == '7':
            print("👋 Bye for now!")
            break
        else:
            

In [ ]:
list_and_summarize_emails(gmail_service)